In [1]:
from dotenv import load_dotenv
from langchain.schema import Document
from operator import itemgetter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.chains.query_constructor.base import AttributeInfo
from langchain_community.document_transformers import LongContextReorder
from langchain.retrievers import SelfQueryRetriever
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import (
    RunnableLambda,
    ConfigurableField,
)
from datetime import datetime
from chromadb.config import Settings

from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import (
    RecursiveJsonSplitter,
)
import json
from tqdm import tqdm
import random
from collections import defaultdict

#### load vectordb

In [3]:
load_dotenv()

True

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

# Chroma 서버 설정
#settings = Settings(chroma_api_impl="chromadb.api.fastapi.FastAPI")
#client = HttpClient(host='localhost', port=8000, settings=settings)

# 이미 저장된 벡터 데이터베이스 불러오기
persist_directory = '../db_date'  # 저장된 데이터베이스 경로
collection_name = 'books_date'  # 사용했던 collection 이름

# 벡터 데이터베이스 로드
vectordb = Chroma(
    collection_name=collection_name,
    persist_directory=persist_directory,
    embedding_function=OpenAIEmbeddings(model='text-embedding-3-small')  # 동일한 임베딩 설정 필요
)


/var/folders/jf/6433sbxs7f5fnynp_fbgf12r0000gn/T/ipykernel_60823/569911102.py:13: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [5]:
# 메타데이터 필드에 대한 정보를 정의합니다.
metadata_field_info = [
    AttributeInfo(
        name='title',  # 출시 연도를 나타내는 필드
        description='책 제목, 책',  # 필드 설명
        type='string',  # 데이터 타입은 정수
    ),
    AttributeInfo(
        name='author',  # 상품 카테고리를 나타내는 필드
        description='작가, 쓴 사람',  # 필드 설명
        type='string',  # 데이터 타입은 문자열
    ),
    AttributeInfo(
        name='publisher',  # 사용자 평점을 나타내는 필드
        description='출판사',  # 필드 설명
        type='string',  # 데이터 타입은 실수
    ),
    AttributeInfo(
        name='pubdate',  # 사용자 평점을 나타내는 필드
        description='출판 연도, 출판일, 출판날',  # 필드 설명
        type='string',  # 데이터 타입은 실수
    ),
]

In [6]:
# SelfQueryRetriever 초기화: LLM과 벡터 저장소를 연결
retriever = SelfQueryRetriever.from_llm(
    llm=ChatOpenAI(model_name='gpt-4o-mini'),  # 사용할 언어 모델 설정
    vectorstore=vectordb,
    document_contents='책 소개',  # 문서 내용 요약
    search_kwargs={'k': 3},  # 검색 결과 개수 설정
    metadata_field_info=metadata_field_info  # 메타데이터 필드 정보
)

In [7]:
def reorder_and_merge_documents(documents):
    # 같은 책(title 기준)으로 문서를 그룹화
    grouped_documents = defaultdict(list)
    for doc in documents:
        title = doc.metadata.get('title', 'N/A')
        grouped_documents[title].append(doc)
    
    # 그룹별로 page_content 병합 및 포맷팅
    merged_documents = []
    for title, docs in grouped_documents.items():
        # 첫 번째 문서의 메타데이터를 사용하고, page_content 병합
        first_doc = docs[0]
        combined_content = ' '.join(doc.page_content.strip() for doc in docs)
        
        # 포맷팅된 문서 생성
        merged_documents.append({
            'title': first_doc.metadata.get('title', 'N/A'),
            'author': first_doc.metadata.get('author', 'N/A'),
            'publisher': first_doc.metadata.get('publisher', 'N/A'),
            'pubdate': first_doc.metadata.get('pubdate', 'N/A'),
            'content': combined_content
        })
        
    #print(merged_documents)
    
    
    # 가독성을 위한 최종 포맷팅
    formatted_documents = '\n\n'.join([
        f"제목: {doc['title']}\n"
        f"작가: {doc['author']}\n"
        f"출판사: {doc['publisher']}\n"
        f"발행일: {doc['pubdate']}\n"
        f"내용: {doc['content']}"
        for doc in merged_documents
    ])
    
    return formatted_documents

# 문서 재정렬 함수 정의
def reorder_documents(documents):
    # LongContextReorder 객체 생성: 긴 문맥을 재정렬하는 기능
    context_reorder = LongContextReorder()
    # 입력된 문서들을 재정렬
    documents_reordered = context_reorder.transform_documents(documents)
    
    # 사람이 볼 수 있도록 각 문서를 정제하여 문자열로 결합
    documents_formatted = '\n\n'.join([
        f"제목: {doc.metadata.get('title', 'N/A')}\n"
        f"작가: {doc.metadata.get('author', 'N/A')}\n"
        f"출판사: {doc.metadata.get('publisher', 'N/A')}\n"
        f"발행연도: {doc.metadata.get('pubdate', 'N/A')}\n"
        f"내용: {doc.page_content}"
        for doc in documents_reordered
    ])
    
    return documents_formatted  # 포맷팅된 문서 내용을 반환

In [8]:
template = '''
너는 책에 대한 정보를 제공하는 봇이야. 
reference에 있는 정보만 사용해서 질문에 답변해야 해. 
reference 외의 내용을 절대 상상하거나 추가하지 마.  
reference에서 정보를 찾을 수 없으면 "요청하신 책에 대한 정보는 도서관에 없어요"라고 답변해야 해.

reference:
{reference}


질문에 답변할 때 다음 규칙을 반드시 따라:
1. reference에서 정보를 찾을 수 없으면 반드시 "요청하신 책에 대한 정보는 도서관에 없어요"라고 답변해야 해.
2. 모든 답변은 아래 형식을 따라야 해:
   제목: [책 제목]  
   작가: [작가 이름]  
   출판사: [출판사 이름]  
   출판연도: [출판 연도]  
   책소개: [책 내용 또는 관련 설명]  
3. 만약 모든 필드를 reference에서 찾을 수 없으면 "정보가 부족합니다."라고 말해야 해.
4. 여러 작품을 추천할 때는 reference에 있는 작품만 포함하고, 나머지는 제외해야 해.

질문: 
{question}

답변은 반드시 한국어로 작성하고, reference에 없는 내용은 절대 추가하지 마.  
만약 reference에 없는 정보를 포함하면 안 된다는 점을 꼭 기억해.
'''

# 템플릿으로부터 프롬프트 생성
prompt = PromptTemplate.from_template(template)

# 모델과 파서 초기화
model = ChatOpenAI(model_name='gpt-4o-mini')
parser = StrOutputParser()

# chain 구성
# chain = (
#     {
#         # 'reference' 키에 대해 여러 처리를 정의
#         'reference': retriever  # SelfQueryRetriever를 통해 reference 추출
#         | RunnableLambda(reorder_documents),  # 문서를 재정렬 및 포맷팅
#         'question': itemgetter('question')  # 질문을 그대로 가져오기
#     }
#     | prompt  # 프롬프트 템플릿에 데이터 결합
#     | model  # 모델에 프롬프트 전달하여 응답 생성
#     | parser  # 모델의 응답을 파싱
# )

chain = (
    {
        # question을 전처리하는 단계 추가
        "question": itemgetter("question"),
        "reference": retriever | RunnableLambda(reorder_documents),
    }
    | prompt  # 템플릿에 데이터 결합
    | model  # 모델 호출
    | parser  # 결과 파싱
)

In [9]:
response = chain.invoke({
    'question': '이레서원에서 나온 책을 알려줘'
})

print(response)


제목: 고린도에서 보낸 일주일 (바울 사역의 사회적, 문화적 정황 이야기)  
작가: 벤 위더링턴 3세  
출판사: 이레서원  
발행연도: 2020  
책소개: “1세기 고린도 사진과 설명을 곁들인 이 흥미로운 이야기는 고린도 서신의 세계를 들여다볼 수 있는 창을 제공한다.”  

제목: 강요된 청빈 (목회자의 경제 현실과 공동체적 극복 방안)  
작가: 정재영  
출판사: 이레서원  
발행연도: 2019  
책소개: 향해서는 이웃과 사회와 작은 교회를 위해 그들이 감당해야 할 책임이 있음을 상기시킨다. 작은 교회를 위해서는, 그들만의 장점을 극대화할 수 있는 목회 방안을 제안한다. 저자의 궁극적인 바람은 교회 규모나 위치, 혹은 교단에 상관없이 모든 교회가 상생하고 협력하는  

제목: 예배학 지도 그리기 (목회자와 예배 사역자를 위한 예배 기획 지침서)  
작가: 문화랑  
출판사: 이레서원  
발행연도: 2020  
책소개: 인간은 하나님을 예배하도록 창조되었다!  


In [28]:
# 작가 
# 이훈구 작가의 책을 알려줘 - 안됨
# 이훈구의 책을 알려줘 - 안됨 
# 작가의 이름이 이훈구인 책을 찾아줘 - 됨

# 출판사
# 이레서원 출판사의 책을 알려줘 - 됨
# 이레서원에서 나온 책을 알려줘 - 됨

# 고양이와 할머니라는 책을 알고싶어 - 됨 
# 고양이와 할머니라는 책은 누가썼어? - 됨


# 기타 
# 음식에 대한 책을 알려줘 - 됨
# 고양이에 대한 책을 알려줘 - 됨
# 경제학에 대한 책을 알려줘 - 됨


# 제목이 "강요된 청빈"인 책을 찾아줘 - 안됨
# 제목이 "강요된 청빈 (목회자의 경제 현실과 공동체적 극복 방안)"인 책을 찾아줘 - 됨

제목: 예배학 지도 그리기  
작가: 문화랑  
출판사: 이레서원  
출판연도: 2020  
책소개: 인간은 하나님을 예배하도록 창조되었다! 이 책에서는 하나님의 백성이 시간과 공간을 정하여 모여서 하나님께 제대로, 아름답게 예배하는 내용과 방법을 살피면서 예배에 대한 역사적 반성과 신학적 고찰을 제시한다. 그리고 예배에서 반복되는 예전(liturgy) 활동이 성도들의 신앙 형성에 어떠한 영향을 미치고, 수 있는 것보다 더 많은 것을 알고 있다”)에 의하여 증명한다. 또한 지적 장애인들의 성찬 참여, 어린이들의 공예배 참여에 대한 숙고 역시 예전의 실천신학적 관점에서 풀어낸다.
